In [106]:
import json
import requests
import argparse
import numpy as np
import pandas as pd
from argparse import Namespace
from datetime import datetime
import pytz

In [107]:
# Give default values so there's no error
default_api_key = ''
args = Namespace(api_key=default_api_key)

API_KEY = '581e043b52c2271b415edaec215732a2' #signed up for a plan from https://api.the-odds-api.com/
SPORT = 'americanfootball_nfl'
REGION = 'us'
MARKET = 'h2h'

In [108]:
odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': API_KEY,
    'sport': SPORT,
    'region': REGION,
    'mkt': MARKET,
})

odds_json = json.loads(odds_response.text)

if not odds_json['success']:
    print(odds_json['msg'])
else:
    print('Number of events:', len(odds_json['data']))
    print(odds_json['data'])

    # Check on my remaining request usage (on free plan)
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 14
[{'id': 'ad2140bcb78fd6f6d37bee937bfa4d90', 'sport_key': 'americanfootball_nfl', 'sport_nice': 'NFL', 'teams': ['Baltimore Ravens', 'Cincinnati Bengals'], 'commence_time': 1700183700, 'home_team': 'Baltimore Ravens', 'sites': [{'site_key': 'fanduel', 'site_nice': 'FanDuel', 'last_update': 1699944845, 'odds': {'h2h': [1.56, 2.5]}}, {'site_key': 'draftkings', 'site_nice': 'DraftKings', 'last_update': 1699944867, 'odds': {'h2h': [1.54, 2.54]}}, {'site_key': 'betus', 'site_nice': 'BetUS', 'last_update': 1699944778, 'odds': {'h2h': [1.53, 2.63]}}, {'site_key': 'lowvig', 'site_nice': 'LowVig.ag', 'last_update': 1699944798, 'odds': {'h2h': [1.53, 2.65]}}, {'site_key': 'betonlineag', 'site_nice': 'BetOnline.ag', 'last_update': 1699944865, 'odds': {'h2h': [1.53, 2.65]}}, {'site_key': 'williamhill_us', 'site_nice': 'Caesars', 'last_update': 1699944882, 'odds': {'h2h': [1.51, 2.62]}}, {'site_key': 'superbook', 'site_nice': 'SuperBook', 'last_update': 1699944869, 'odds': {'h2h

In [109]:
def convertTime(unixTimeStamp):
    # Convert timestamp to datetime object in UTC
    date_time_utc = datetime.utcfromtimestamp(unixTimeStamp)

    # Specify the UTC time zone
    utc_timezone = pytz.timezone('UTC')

    #Localize the UTC datetime object
    date_time_utc = utc_timezone.localize(date_time_utc)

    # Convert to Eastern Standard Time (EST)
    est_timezone = pytz.timezone('US/Eastern')
    date_time_est = date_time_utc.astimezone(est_timezone)

    formatted_date_est = date_time_est.strftime('%Y-%m-%d %H:%M:%S %Z')

    return formatted_date_est

In [110]:
def decimal_to_american(decimal_odds):
    if decimal_odds == 1.0:
        # This is only the case in extreme blowouts, and you are no longer able to bet
        return None
    elif decimal_odds >= 2.0:
        # Add a "+" ahead of positive numbers
        return f"+{round((decimal_odds - 1) * 100)}"
    else:
        return round(-100 / (decimal_odds - 1))

In [111]:
print("Betting Information for this week's NFL Football Games: Moneyline Odds")
print("-------------------------------------------------------")
games = str(len(odds_json['data']))
print(games + " total NFL game betting lines")

gameID = []
team = []
moneyline = []
lastUpdate = []

count = 1
for game in odds_json['data']:
    team1 = game['teams'][0]
    team2 = game['teams'][1]
    print("Game: " + team1 + " vs. " + team2)

    
    sites = game['sites']
    for site_data in sites:
        if site_data['site_key'] == 'fanduel':
            fanduel_update = convertTime(site_data['last_update'])
            print("\tFanduel odds (Last update: " + str(fanduel_update) + "")
            
            fanduel_odds = site_data['odds']['h2h']
            fanduel_odds_american = [decimal_to_american(odds) for odds in fanduel_odds]
            team1odds = fanduel_odds_american[0]
            team2odds = fanduel_odds_american[1]

            print("\t\t" + team1 + ": " + str(team1odds))
            print("\t\t" + team2 + ": " + str(team2odds))

            #Add to lists to make a df
            gameID.append(count)
            gameID.append(count)
            count = count + 1
            
            moneyline.append(team1odds)
            moneyline.append(team2odds)

            lastUpdate.append(fanduel_update)
            lastUpdate.append(fanduel_update)

            team.append(team1)
            team.append(team2)
     
        elif site_data['site_key'] == 'draftkings':
            draftkings_update = convertTime(site_data['last_update'])
            print("\tDraftKings odds (Last update: " + str(draftkings_update) + ")")
            
            draftkings_odds = site_data['odds']['h2h']
            draftkings_odds_american = [decimal_to_american(odds) for odds in draftkings_odds]
            team1odds = draftkings_odds_american[0]
            team2odds = draftkings_odds_american[1]


            print("\t\t" + team1 + ": " + str(team1odds))
            print("\t\t" + team2 + ": " + str(team2odds))


Betting Information for this week's NFL Football Games: Moneyline Odds
-------------------------------------------------------
14 total NFL game betting lines
Game: Baltimore Ravens vs. Cincinnati Bengals
	Fanduel odds (Last update: 2023-11-14 01:54:05 EST
		Baltimore Ravens: -179
		Cincinnati Bengals: +150
	DraftKings odds (Last update: 2023-11-14 01:54:27 EST)
		Baltimore Ravens: -185
		Cincinnati Bengals: +154
Game: Green Bay Packers vs. Los Angeles Chargers
	Fanduel odds (Last update: 2023-11-14 01:54:05 EST
		Green Bay Packers: +138
		Los Angeles Chargers: -164
	DraftKings odds (Last update: 2023-11-14 01:54:27 EST)
		Green Bay Packers: +142
		Los Angeles Chargers: -169
Game: Jacksonville Jaguars vs. Tennessee Titans
	Fanduel odds (Last update: 2023-11-14 01:54:05 EST
		Jacksonville Jaguars: -294
		Tennessee Titans: +240
	DraftKings odds (Last update: 2023-11-14 01:54:27 EST)
		Jacksonville Jaguars: -263
		Tennessee Titans: +215
Game: Las Vegas Raiders vs. Miami Dolphins
	Fanduel 

In [112]:
fanduel_df = pd.DataFrame({'gameID':gameID, 'team': team, 'Last Update': lastUpdate, 'moneyline': moneyline})
fanduel_df

,gameID,team,Last Update,moneyline
0,1,Baltimore Ravens,2023-11-14 01:54:05 EST,-179
1,1,Cincinnati Bengals,2023-11-14 01:54:05 EST,+150
2,2,Green Bay Packers,2023-11-14 01:54:05 EST,+138
3,2,Los Angeles Chargers,2023-11-14 01:54:05 EST,-164
4,3,Jacksonville Jaguars,2023-11-14 01:54:05 EST,-294
5,3,Tennessee Titans,2023-11-14 01:54:05 EST,+240
6,4,Las Vegas Raiders,2023-11-14 01:54:05 EST,+460
7,4,Miami Dolphins,2023-11-14 01:54:05 EST,-625
8,5,Carolina Panthers,2023-11-14 01:54:05 EST,+460
9,5,Dallas Cowboys,2023-11-14 01:54:05 EST,-625


In [104]:
fanduel_df.dtypes

gameID          int64
team           object
Last Update    object
moneyline      object
dtype: object

In [105]:
fanduel_df['Last Update'] = pd.to_datetime(fanduel_df['Last Update'])

#gets rid of '+' sign in front of positive numbers
fanduel_df['moneyline'] = pd.to_numeric(fanduel_df['moneyline'], errors='coerce').fillna(0).astype(int)

#Sort by biggest underdogs
fanduel_df.sort_values("moneyline", ascending=False)

/var/folders/cj/jcn0rhn52g397xd7gy7ndwnc0000gn/T/ipykernel_70758/322050936.py:1: FutureWarning: Parsing 'EST' as tzlocal (dependent on system timezone) is deprecated and will raise in a future version. Pass the 'tz' keyword or call tz_localize after construction instead
  fanduel_df['Last Update'] = pd.to_datetime(fanduel_df['Last Update'])


,gameID,team,Last Update,moneyline
14,8,Carolina Panthers,2023-11-12 19:40:50-05:00,410
25,13,Tampa Bay Buccaneers,2023-11-12 19:40:50-05:00,385
12,7,Las Vegas Raiders,2023-11-12 19:40:50-05:00,380
20,11,New York Giants,2023-11-12 19:40:50-05:00,360
22,12,Chicago Bears,2023-11-12 19:40:50-05:00,330
1,1,Washington Commanders,2023-11-12 19:36:28-05:00,320
5,3,Denver Broncos,2023-11-12 19:40:50-05:00,270
27,14,New York Jets,2023-11-12 19:40:50-05:00,250
11,6,Tennessee Titans,2023-11-12 19:40:50-05:00,220
16,9,Arizona Cardinals,2023-11-12 19:40:50-05:00,200
